# An Exploration of Misleading Data Visualization Using Toronto Immunization Coverage Data

In this report, my aim is to show some infovisualizations derived from immunization data from Toronto's Open Data Portal, in order to demonstrate how manipulations with leading language, visual color cues, and misrepresentation of data can mislead viewers of such graphics.

## Cherrypicking data

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import folium
from shapely.geometry import Point, Polygon

In [2]:
df_2019 = pd.read_csv("immunization-coverage-2018-2019.csv")
df_2019.head(10)

,_id,School Name,Enrolled population,DTP coverage rate (%),DTP Religious exemption rate (%),MMR coverage rate (%),MMR Religious exemption rate (%),Lat,Lng
0,1,A Y Jackson S.S.,1027,89.0,1.1,96.5,1.1,43.805261,-79.366555
1,2,Academie Alexandre-Dumas,129,86.8,1.6,89.1,1.6,43.762419,-79.179765
2,3,Adam Beck Jr P.S.,309,96.1,3.6,95.8,3.6,43.683152,-79.288488
3,4,Africentric Alternative School,77,71.4,20.8,72.7,20.8,43.745424,-79.488261
4,5,Agincourt C.I.,1241,87.3,1.0,98.1,1.0,43.788874,-79.278910
5,6,Agincourt Jr P.S.,141,99.3,0.0,98.6,0.0,43.788406,-79.281172
6,7,Agnes MacPhail P.S.,207,94.7,0.0,96.6,0.0,43.819269,-79.278553
7,8,Albert Campbell C.I.,1072,88.2,0.4,96.8,0.4,43.809014,-79.273407
8,9,Albion Heights JMS,294,93.9,0.7,97.3,1.0,43.731388,-79.594316
9,10,Alexander Muir/Gladstone Ave Jr & Sr P.S.,324,90.7,1.9,94.4,1.9,43.646021,-79.428847


In [3]:
# Extract data points where either the DTP or MMR religious exemption rates from immunization are greater than 10%
high_rer_rate_schools = df_2019[df_2019["DTP Religious exemption rate (%)" or "MMR Religious exemption rate (%)"] >= 10]
len(high_rer_rate_schools)

12

In [4]:
map1_points = high_rer_rate_schools[['Lat','Lng']]
map1_list = high_rer_rate_schools[['Lat','Lng']].values.tolist()
map1 = folium.Map(location=[43.6532,-79.3832], tiles='Stamen Toner', zoom_start=11)
for point in range(0, len(map1_points)):
    folium.CircleMarker(map1_list[point], radius=15, color='red', fill=True).add_to(map1)
map1

<h3><center>1. More than 10% of children in these Toronto schools are not immunized</center></h3>

The combination of language used in the graphic's title, to the red color of its markers, and the underlying manipulations of what datapoints are represented, altogether contribute to an alarmist message. Out of context, this graphic could be used to distress parents considering different schools for their children.
With a group of these points concentrated in a cluster downtown, these markers may send messages to some parents that there is a greater prevalence of immunization exemptions in these areas than they may have previously thought. 

However, **the map does not show all schools listed in the original data**, so viewers have no way to explore the proportion of schools represented here among all schools in Toronto. In graphic 2, it can be seen that the red cluster of alarm in graphic 1 is likely the result of the greater number of schools located downtown. 

Furthermore, all schools are shown as equal data points, with no information regarding the size of each school population. **The enrolled population at these schools is not considered along with their immunization rates**. 

Lastly, **the data is not specified - both MMR and DTP immunization exemption rates have been considered**, and if one of them is over 10% it is included with no indication as to whether it is the MMR religious exemption rate or DTP religious exemption rate that is being used for the school's inclusion within the graphic. DTP coverage refers to the rate of immunization for diphtheria, tetanus, and polio. MMR coverage refers to the rate of immunization for measles, mumps, and rubella.

To provide a more contextualized look at the data, the following revision shows what this visualization would look like with all schools from the original dataset included (but maintaining both the lack of representation of enrolled population, and the lack of specificity with regards to which immunization is being considered).

In [5]:
# Retrieve all other data points of original dataset
other_schools = df_2019[df_2019["DTP Religious exemption rate (%)" or "MMR Religious exemption rate (%)"] < 10]

In [6]:
map2_points = other_schools[['Lat','Lng']]
map2_list = other_schools[['Lat','Lng']].values.tolist()
for point in range(0, len(map2_points)):
    folium.CircleMarker(map2_list[point], radius=15, color='blue', fill=True).add_to(map1)
map1

<h3><center>2. Red circles represent Toronto schools with more than a 10% religious exemption rate for either DTP or MMR immunization</center></h3>

With more accurate titling, and inclusion of all datapoints from the original dataset, much of the alarmist impact of the first graphic is reduced.

## Misrepresentation of data 

To explore an even more dubious approach to this dataset, in the following steps I have created a new "measure", the "Secondary DTP Exemption Rate %", named such because it is derived secondarily based on calculations on the primary data. This measure is created by subtracting the original dataset's measure of "DTP coverage rate %" from 100%. 

In [7]:
# Manufacturing a measure of exemption from DTP immunization, based on coverage rate
df_2019["Secondary DTP Exemption %"] = 100 - df_2019["DTP coverage rate (%)"]

# Selecting schools with a manufactured DTP exemption rate over 10%
high_dtp_exemption_rate_schools = df_2019[df_2019["Secondary DTP Exemption %"] > 10]
len(high_dtp_exemption_rate_schools)

377

In [8]:
map3_points = high_dtp_exemption_rate_schools[['Lat','Lng']]
map3_list = high_dtp_exemption_rate_schools[['Lat','Lng']].values.tolist()
map3 = folium.Map(location=[43.6932,-79.3872], tiles='Stamen Toner', zoom_start=12)
for point in range(0, len(map3_points)):
    folium.CircleMarker(map3_list[point], radius=15, color='red', fill=True).add_to(map3)
map3

<h6><div style="text-align: right">*calculated from <i>(100% - 2019 DTP coverage %)</i> </div></h6>
<h3><center>3. Red hotspots represent Toronto schools with more than 10% children not immunized for DTP</center></h3>

An *even more* questionable approach has been taken in creating graphic 3.

The original data from which the graphic was created has been manipulated, but its manipulation has been reduced to an asterisk, which indicates that the unimmunized rate has been calculated from the immunization coverage rate. While this might seem reasonable at first, **this rate is completely artificial, as this column does not exist in the original data**. The original data provides an immunization coverage rate, and a religious exemption rate. These proportions do not cover the entire student population, with some remaining percentage who may be exempt for medical or administrative reasons, or who are not complete for age for immunization. This information is found in the immunization data's README file.

**Creating an artificial rate from existing data, and combining different kinds of exemptions in the process, creates a misleading picture of the original data.** This would exemplify unethical data practice.

These map visualizations, if interrogated with some of Darrell Huff's famous questions - specifically, *what's missing? did somebody change the subject?* - would be uncovered to be deceptive, as both use biased selection processes, as well as language and visual cues reminiscent of "hotspots" to attempt to manipulate the viewer's emotional response. Open data processes, including the sharing of data processing methods for visualization, are integral to combat such manipulations in information visualization.

## References

Huff, D. (1954). How to talk back to a statistic. In <i>How to lie with statistics</i>. 

Open Toronto. (2019). <i>Immunization coverage for students 2018-2019</i> [Data set]. https://open.toronto.ca/dataset/immunization-coverage-for-students/